In [1]:
# Import used packages
import geopandas as gpd  # used to read the shapfile
import rasterio as rio   # used to read the raster (.tif) files
from rasterio.plot import show # used to make plots using rasterio
import matplotlib.pyplot as plt #to make plots using matplotlib
import pandas as pd
import os
import numpy as np
from rasterio.warp import calculate_default_transform, reproject, Resampling
import glob
import rasterio

In [2]:
os.chdir(r'C:\Users\hdey\Downloads\HimFac\PCA_USGAC')

In [ ]:
#Filtering -999 values
# Define the input and output file paths
input_raster = "DEM_USGAC_250m.tif"
output_raster = "DEM_USGAC_250m_filter.tif"

# Define the value range to change
#min_value = -999  # Lower bound (inclusive)
#max_value = 0  # Upper bound (inclusive)

# Open the raster dataset
with rio.open(input_raster) as src:
    data = src.read(1)  # Read the first band
    profile = src.profile  # Get the metadata

# Replace values within the range with zero
data[data < 0] = 0

# Write the modified raster to a new file
with rio.open(output_raster, "w", **profile) as dst:
    dst.write(data, 1)

print("Raster modification complete. Saved as", output_raster)

In [ ]:
import rasterio as rio
import geopandas as gpd
from rasterio.mask import mask
import numpy as np
from scipy.ndimage import distance_transform_edt

# File paths
input_raster = "NoHSD.tif"
output_raster = "NoHSD_Fvoid.tif"
boundary_shp = "US_Gulf_Atlantic_Coast_100MilesBuffer.shp"

# Load boundary shapefile
gdf = gpd.read_file(boundary_shp)
geometry = gdf.geometry.values

# Open raster
with rio.open(input_raster) as src:
    data = src.read(1)
    profile = src.profile
    nodata = src.nodata

    # Mask the raster to get the buffer area (same size as original)
    masked_data, _ = mask(src, geometry, crop=False)
    masked_data = masked_data[0]  # Get 2D array

# Create a mask: True where we want to fill (inside buffer AND is NoData)
if nodata is None:
    nodata = -999
    data[data == nodata] = np.nan

fill_mask = np.isnan(data) & (masked_data != nodata)

# Prepare data for interpolation: keep valid values, set others to np.nan
data_with_nan = np.where(data == nodata, np.nan, data)

# Use distance transform to get nearest non-NaN values
# Create inverse mask (True = valid values)
valid_mask = ~np.isnan(data_with_nan)
dist, indices = distance_transform_edt(valid_mask == 0,
                                       return_distances=True,
                                       return_indices=True)

# Create filled array using nearest neighbor interpolation
filled_data = data_with_nan.copy()
filled_data[fill_mask] = data_with_nan[indices[0][fill_mask], indices[1][fill_mask]]

# Replace remaining NaNs outside buffer (optional: leave them as-is)
filled_data[np.isnan(filled_data)] = nodata

# Update metadata
profile.update(nodata=nodata)

# Save output raster
with rio.open(output_raster, 'w', **profile) as dst:
    dst.write(filled_data, 1)

print("Voids filled with nearest values within the boundary. Output saved to", output_raster)



In [5]:
# Read your point shapefiles (Flooded and Non Flooded locations)
points=gpd.read_file('Destroyed_FL_dmg19k.shp')

In [6]:
#Match their GCS
with rio.open("Elevation.tif") as El_raster:
    El_arr = El_raster.read(1)  # Read as array
    transform_func = El_raster.transform  # Raster transformation matrix 
    raster_crs = El_raster.crs  # Get CRS of raster

    # Reproject points to match raster CRS
    points = points.to_crs(raster_crs)

In [ ]:
import os
import glob
import numpy as np
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

# Define the reference raster
reference_raster = r"C:\Users\hdey\Downloads\HimFac\OriReprojection\Elevation.tif"

# Get a list of target rasters (Fixing the glob path)
target_rasters = glob.glob(r"C:\Users\hdey\Downloads\HimFac\OriReprojection\*.tif")

# Create an output folder for reprojected rasters
output_folder = r"C:\Users\hdey\Downloads\HimFac\OriReprojection\FinalRepj"
os.makedirs(output_folder, exist_ok=True)

# Open the reference raster to get CRS, transform, and resolution
with rasterio.open(reference_raster) as ref:
    ref_crs = ref.crs  # CRS of reference raster
    ref_transform = ref.transform  # Reference transform
    ref_width = ref.width  # Reference raster width (columns)
    ref_height = ref.height  # Reference raster height (rows)
    ref_resolution = ref.res  # Reference resolution (tuple)
    ref_bounds = ref.bounds  # Reference extent

# Loop through each target raster to reproject and resample
for raster in target_rasters:
    with rasterio.open(raster) as src:
        src_crs = src.crs  # Original CRS
        src_transform = src.transform  # Original transform
        src_data = src.read(1)  # Read raster band (assuming single-band)

        # Handle NoData and ensure dtype consistency
        nodata_value = src.nodata if src.nodata is not None else -9999
        
        # Ensure dtype consistency (convert to float to handle NaN safely)
        if np.issubdtype(src_data.dtype, np.integer):
            src_data = src_data.astype('float32')
        
        # Replace NoData with NaN for processing
        src_data[src_data == nodata_value] = np.nan
        
        # Check if reprojection or resampling is needed
        if src_crs != ref_crs or src.res != ref_resolution:
            print(f"🔄 Reprojecting and Resampling: {os.path.basename(raster)}")

            # Use the reference raster's transform, width, and height
            dst_transform = ref_transform
            dst_width = ref_width
            dst_height = ref_height

            # Create an empty array for the reprojected data
            dst_data = np.empty((dst_height, dst_width), dtype='float32')

            # Reproject and resample the raster to match reference
            reproject(
                source=src_data,
                destination=dst_data,
                src_transform=src_transform,
                src_crs=src_crs,
                dst_transform=dst_transform,
                dst_crs=ref_crs,
                resampling=Resampling.bilinear  # Use bilinear resampling for smooth results
            )
        else:
            print(f"✅ {os.path.basename(raster)} already matches CRS and resolution.")
            dst_data = src_data
            dst_transform = src_transform
            dst_width = src.width
            dst_height = src.height

        # Replace NaN with original NoData value before saving
        dst_data[np.isnan(dst_data)] = nodata_value

        # Update metadata
        profile = src.profile
        profile.update(
            dtype='float32',  # Use float to handle NaN correctly
            crs=ref_crs,
            transform=dst_transform,
            width=dst_width,
            height=dst_height,
            nodata=nodata_value  # Maintain original NoData value
        )

        # Save the reprojected and resampled raster
        output_raster = os.path.join(output_folder, os.path.basename(raster))
        with rasterio.open(output_raster, "w", **profile) as dst:
            dst.write(dst_data, 1)

        print(f"✅ Saved: {output_raster}")




In [7]:
# make columns to extract the values of each predictive feature
# for each point. 
points['Elevation']=0 #
points['Slope']=0
points['Soil_Moisture']=0
points['Precip91']=0
points['DisRiv']=0
points['NDVI']=0
points['TWI']=0
points['DD']=0
points['PopDen']=0
points['BuildingHeight']=0
points['Dis_Road']=0
#points['F1_POV_NH_L_M']=0
#points['F2_Dis_MH_65']=0
#points['F3_Fem_05']=0
#points['Female']=0
points['POV']=0
points['Minority']=0
points['NoHSD']=0
points['Age65']=0
points['Age5']=0
#points['Unemployed']=0

In [8]:
#The predictive features are in raster format so we use rasterio package to 
#read them and convert them to numpy array

El_raster=rio.open("Elevation.tif")
El_arr=El_raster.read(1)

Slope_raster=rio.open("Slope.tif")
Slope_arr=Slope_raster.read(1)

Soil_Moisture_raster=rio.open("Soil_Moisture.tif")
Soil_Moisture_arr=Soil_Moisture_raster.read(1)

Precip91_raster=rio.open("Precip91.tif")
Precip91_arr=Precip91_raster.read(1)

DisRiv_raster=rio.open("DisRiv.tif")
DisRiv_arr=DisRiv_raster.read(1)

NDVI_raster=rio.open("NDVI.tif")
NDVI_arr=NDVI_raster.read(1)

TWI_raster=rio.open("TWI.tif")
TWI_arr=TWI_raster.read(1)

DD_raster=rio.open("DD.tif")
DD_arr=DD_raster.read(1)

PopDen_raster=rio.open("PopDen.tif")
PopDen_arr=PopDen_raster.read(1)

BuildingHeight_raster=rio.open("BuildingHeight.tif")
BuildingHeight_arr=BuildingHeight_raster.read(1)

Dis_Road_raster=rio.open("Dis_Road.tif")
Dis_Road_arr=Dis_Road_raster.read(1)

#F1_POV_NH_L_M_raster=rio.open("F1_POV_NH_L_M.tif")
#F1_POV_NH_L_M_arr=F1_POV_NH_L_M_raster.read(1)

#F2_Dis_MH_65_raster=rio.open("F2_Dis_MH_65.tif")
#F2_Dis_MH_65_arr=F2_Dis_MH_65_raster.read(1)

#F3_Fem_05_raster=rio.open("F3_Fem_05.tif")
#F3_Fem_05_arr=F3_Fem_05_raster.read(1)

#Female_raster=rio.open("Female.tif")
#Female_arr=Female_raster.read(1)

POV_raster=rio.open("POV.tif")
POV_arr=POV_raster.read(1)

Minority_raster=rio.open("Minority.tif")
Minority_arr=Minority_raster.read(1)


NoHSD_raster=rio.open("NoHSD.tif")
NoHSD_arr=NoHSD_raster.read(1)


Age65_raster=rio.open("Age65.tif")
Age65_arr=Age65_raster.read(1)


Age5_raster=rio.open("Age5.tif")
Age5_arr=Age5_raster.read(1)

#Unemployed_raster=rio.open("Unemployed.tif")
#Unemployed_arr=Unemployed_raster.read(1)


In [9]:
print(f"El_arr.shape: {El_arr.shape}")
print(f"Slope_arr.shape: {Slope_arr.shape}")
print(f"Precip91_arr.shape: {Precip91_arr.shape}")
print(f"NDVI_arr.shape: {NDVI_arr.shape}")
print(f"DD_arr.shape: {DD_arr.shape}")
print(f"Soil_Moisture_arr.shape: {Soil_Moisture_arr.shape}")
print(f"DisRiv_arr.shape: {DisRiv_arr.shape}")
print(f"TWI_arr.shape: {TWI_arr.shape}")
print(f"PopDen_arr.shape: {PopDen_arr.shape}")
print(f"BuildingHeight_arr.shape: {BuildingHeight_arr.shape}")
print(f"Dis_Road_arr.shape: {Dis_Road_arr.shape}")
#print(f"Female_arr.shape: {Female_arr.shape}")
print(f"Age5_arr.shape: {Age5_arr.shape}")
print(f"Age65_arr.shape: {Age65_arr.shape}")
#print(f"Unemployed_arr.shape: {Unemployed_arr.shape}")
print(f"Minority_arr.shape: {Minority_arr.shape}")
print(f"POV_arr.shape: {POV_arr.shape}")
print(f"NoHSD_arr.shape: {NoHSD_arr.shape}")
#print(f"F1_POV_NH_L_M_arr.shape: {F1_POV_NH_L_M_arr.shape}")
#print(f"F2_Dis_MH_65_arr.shape: {F2_Dis_MH_65_arr.shape}")
#print(f"F3_Fem_05_arr.shape: {F3_Fem_05_arr.shape}")



El_arr.shape: (8806, 12979)
Slope_arr.shape: (8806, 12979)
Precip91_arr.shape: (8806, 12979)
NDVI_arr.shape: (8806, 12979)
DD_arr.shape: (8806, 12979)
Soil_Moisture_arr.shape: (8806, 12979)
DisRiv_arr.shape: (8806, 12979)
TWI_arr.shape: (8806, 12979)
PopDen_arr.shape: (8806, 12979)
BuildingHeight_arr.shape: (8806, 12979)
Dis_Road_arr.shape: (8806, 12979)
Age5_arr.shape: (8806, 12980)
Age65_arr.shape: (8806, 12980)
Minority_arr.shape: (8806, 12980)
POV_arr.shape: (8806, 12980)
NoHSD_arr.shape: (8806, 12980)


In [ ]:
#show point and raster on a matplotlib plot
fig, ax = plt.subplots(figsize=(30,30))
#points.plot(ax=ax, cmap='terrain')
show(DD_raster, ax=ax)

In [10]:
# Extracting the raster values to the points shapefile
# count=0
for index, row in points.iterrows():  # iterate over the points in the shapefile
    longitude = row['geometry'].x  # get the longitude of the point
    latitude = row['geometry'].y  # get the latitude of the point

    # Print the longitude and latitude to debug
    print(f"Longitude: {longitude}, Latitude: {latitude}")

    # Get the corresponding pixel to the point (longitude, latitude)
    try:
        rowIndex, colIndex = El_raster.index(longitude, latitude)
    except Exception as e:
        print(f"Error finding indices for point {index}: {e}")
        continue  # Skip this point if there's an error

    # Check if the row and column indices are within the bounds of the raster
    if 0 <= rowIndex < El_arr.shape[0] and 0 <= colIndex < El_arr.shape[1]:
        # Extract the raster values at the point location
        points['Elevation'].loc[index] = El_arr[rowIndex, colIndex]
        points['Slope'].loc[index] = Slope_arr[rowIndex, colIndex]
        points['Soil_Moisture'].loc[index] = Soil_Moisture_arr[rowIndex, colIndex]
        points['Precip91'].loc[index] = Precip91_arr[rowIndex, colIndex]
        points['DisRiv'].loc[index] = DisRiv_arr[rowIndex, colIndex]
        points['NDVI'].loc[index] = NDVI_arr[rowIndex, colIndex]
        points['TWI'].loc[index] = TWI_arr[rowIndex, colIndex]
        points['DD'].loc[index] = DD_arr[rowIndex, colIndex]
        points['PopDen'].loc[index] = PopDen_arr[rowIndex, colIndex]
        points['BuildingHeight'].loc[index] = BuildingHeight_arr[rowIndex, colIndex]
        points['Dis_Road'].loc[index] = Dis_Road_arr[rowIndex, colIndex]
        #points['F1_POV_NH_L_M'].loc[index] = F1_POV_NH_L_M_arr[rowIndex, colIndex]
        #points['F2_Dis_MH_65'].loc[index] = F2_Dis_MH_65_arr[rowIndex, colIndex]
        #points['F3_Fem_05'].loc[index] = F3_Fem_05_arr[rowIndex, colIndex]
        #points['Female'].loc[index] = Female_arr[rowIndex, colIndex]
        points['POV'].loc[index] = POV_arr[rowIndex, colIndex]
        points['Minority'].loc[index] = Minority_arr[rowIndex, colIndex]
        points['NoHSD'].loc[index] = NoHSD_arr[rowIndex, colIndex]
        points['Age65'].loc[index] = Age65_arr[rowIndex, colIndex]
        points['Age5'].loc[index] = Age5_arr[rowIndex, colIndex]
        #points['Unemployed'].loc[index] = Unemployed_arr[rowIndex, colIndex]
    else:
        # Log the out-of-bounds issue with the index and raster shape
        print(f"Point at index {index} with coordinates (Longitude: {longitude}, Latitude: {latitude}) is out of bounds for the raster.")


    

Longitude: -81.38525841599994, Latitude: 25.861552072000052
Longitude: -81.38454922799997, Latitude: 25.861333218000027
Longitude: -81.38538098099997, Latitude: 25.861311518000036
Longitude: -81.38333812899998, Latitude: 25.860912984000038
Longitude: -81.38531792499998, Latitude: 25.860772635000046
Longitude: -81.38455770999997, Latitude: 25.860499659000027
Longitude: -81.38531185099998, Latitude: 25.86049597300007
Longitude: -81.38456053299996, Latitude: 25.86022181100003
Longitude: -81.38536970199993, Latitude: 25.859943223000073
Longitude: -81.38456338499998, Latitude: 25.85994138600006
Longitude: -81.38230558299995, Latitude: 25.857981154000072
Longitude: -81.38611574199996, Latitude: 25.859380955000063
Longitude: -81.38559728599995, Latitude: 25.859479107000027
Longitude: -81.38526427499994, Latitude: 25.859476534000066
Longitude: -81.37164270899996, Latitude: 25.845720800000038
Longitude: -81.37067359899999, Latitude: 25.84568844200004
Longitude: -81.37258748999994, Latitude: 25.

C:\Users\hdey\AppData\Local\Temp\ipykernel_27072\370351802.py:20: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  points['Elevation'].loc[index] = El_arr[rowIndex, colIndex]
C:\Users\hdey\AppData\Local\Temp\ipykernel_27072\370351802.py:20: Set

Longitude: -82.45351867799997, Latitude: 27.934197391000055
Longitude: -82.45298723799993, Latitude: 27.933216428000037
Longitude: -82.45296442499995, Latitude: 27.933167403000027
Longitude: -82.45394528799994, Latitude: 27.934966849000034
Longitude: -82.45392135899993, Latitude: 27.934923756000046
Longitude: -82.45509169499996, Latitude: 27.93711027100005
Longitude: -82.45463908899995, Latitude: 27.936232852000046
Longitude: -82.45377654599997, Latitude: 27.93466253200006
Longitude: -82.45337161399993, Latitude: 27.933932241000036
Longitude: -82.45446086999993, Latitude: 27.935906884000076
Longitude: -82.45443540299993, Latitude: 27.935860319000028
Longitude: -82.45301094599995, Latitude: 27.933267396000076
Longitude: -82.45379876299995, Latitude: 27.934702621000042
Longitude: -82.45499537199998, Latitude: 27.936884406000047
Longitude: -82.45496990099997, Latitude: 27.936837850000074
Longitude: -82.45375432199995, Latitude: 27.934622438000076
Longitude: -82.45373209399997, Latitude: 2

C:\Users\hdey\AppData\Local\Temp\ipykernel_27072\370351802.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '9.344148566015065e-09' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  points['PopDen'].loc[index] = PopDen_arr[rowIndex, colIndex]


Longitude: -96.00554617799997, Latitude: 29.829922060000058
Longitude: -96.08753845899997, Latitude: 29.810314515000073
Longitude: -96.02791651499996, Latitude: 29.811243501000035
Longitude: -96.01142158199997, Latitude: 29.789777152000056
Longitude: -95.98846421299999, Latitude: 29.790675056000055
Longitude: -96.01869195399996, Latitude: 29.820224945000064
Longitude: -96.00267523399998, Latitude: 29.78783405300004
Longitude: -95.99660493199997, Latitude: 29.787665227000048
Longitude: -96.09189612899996, Latitude: 29.963006999000072
Longitude: -96.03140477399995, Latitude: 29.83320163600007
Longitude: -96.03289244699994, Latitude: 29.83239498300003
Longitude: -96.01547646499995, Latitude: 29.833812358000046
Longitude: -96.00447795699995, Latitude: 29.841409070000054
Longitude: -96.02563618299996, Latitude: 29.83025500900004
Longitude: -96.01285455199996, Latitude: 29.818361141000025
Longitude: -96.02050046499994, Latitude: 29.819223993000037
Longitude: -96.04998829199997, Latitude: 29.

In [ ]:
print(El_arr.shape)  # For NumPy arrays
print(Age5_raster.shape)  # For Pandas DataFrame


In [11]:
points.head()

,class,geometry,Elevation,Slope,Soil_Moisture,Precip91,DisRiv,NDVI,TWI,DD,PopDen,BuildingHeight,Dis_Road,POV,Minority,NoHSD,Age65,Age5
0,1,POINT (-81.38526 25.86155),5,0.477624,38,119.992676,253.776978,0.431880,1.154943,1.502872,2.0,1,0.054974,21.4,9.2,4.7,35.6,5
1,1,POINT (-81.38455 25.86133),-2,0.190525,38,120.260002,247.574829,0.667213,2.244427,1.505790,2.0,1,0.054001,5.3,1.5,0.8,13.3,0
2,1,POINT (-81.38538 25.86131),5,0.477624,38,119.992676,253.776978,0.431880,1.154943,1.502872,2.0,1,0.054974,21.4,9.2,4.7,35.6,5
3,1,POINT (-81.38334 25.86091),-2,0.190525,38,120.260002,247.574829,0.667213,2.244427,1.505790,2.0,1,0.054001,5.3,1.5,0.8,13.3,0
4,1,POINT (-81.38532 25.86077),5,0.477624,38,119.992676,253.776978,0.431880,1.154943,1.502872,2.0,1,0.054974,21.4,9.2,4.7,35.6,5


In [12]:
df = pd.DataFrame(points)
df

,class,geometry,Elevation,Slope,Soil_Moisture,Precip91,DisRiv,NDVI,TWI,DD,PopDen,BuildingHeight,Dis_Road,POV,Minority,NoHSD,Age65,Age5
0,1,POINT (-81.38526 25.86155),5,0.477624,38,119.992676,253.776978,0.431880,1.154943,1.502872,2.0,1,0.054974,21.4,9.2,4.7,35.6,5
1,1,POINT (-81.38455 25.86133),-2,0.190525,38,120.260002,247.574829,0.667213,2.244427,1.505790,2.0,1,0.054001,5.3,1.5,0.8,13.3,0
2,1,POINT (-81.38538 25.86131),5,0.477624,38,119.992676,253.776978,0.431880,1.154943,1.502872,2.0,1,0.054974,21.4,9.2,4.7,35.6,5
3,1,POINT (-81.38334 25.86091),-2,0.190525,38,120.260002,247.574829,0.667213,2.244427,1.505790,2.0,1,0.054001,5.3,1.5,0.8,13.3,0
4,1,POINT (-81.38532 25.86077),5,0.477624,38,119.992676,253.776978,0.431880,1.154943,1.502872,2.0,1,0.054974,21.4,9.2,4.7,35.6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38539,0,POINT (-70.98972 42.50763),51,2.810176,18,104.524841,3956.260010,0.470408,-1.696088,1.524060,1591.0,2,0.001575,8.7,14.3,3.1,16.6,7
38540,0,POINT (-67.94204 45.07985),97,2.686046,24,99.719170,2479.779297,0.622738,-0.132688,2.032392,2.0,1,0.190662,36.9,30.6,10.0,25.0,5
38541,0,POINT (-96.78167 29.98370),136,0.476802,19,85.611420,4205.704590,0.466418,2.301585,1.682807,15.0,1,0.015767,18.9,11.2,7.0,35.1,1
38542,0,POINT (-88.64574 31.49805),53,0.174935,31,128.416931,9121.952148,0.691004,8.509232,3.259017,11.0,1,0.094692,14.8,2.0,14.4,15.2,2


In [13]:
df.to_csv('Destroyed_FL_dmg19k.csv', index=False) 